In [1]:
import warnings
warnings.filterwarnings("ignore")

* Bedrock client

In [3]:
import os
import boto3
from dotenv import load_dotenv
load_dotenv(override=True)

region_name = os.getenv("AWS_LLM_REGION_NAME")
model_id = os.getenv("AWS_LLM_MODEL_ID") 

#Create the Bedrock client and the Bedrock LLM
client = boto3.client(service_name="bedrock-runtime", region_name=region_name)


* Bedrock llm - langchain lib

In [4]:
from langchain_aws import ChatBedrockConverse

bedrock_llm=  ChatBedrockConverse(
                model= model_id ,
                temperature=0,
                max_tokens=None,
                client=client)

In [5]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "You are an assistant for question-answering tasks.Use three sentences maximum and keep the answer concise."
user_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
            ("system",system_template),
            ("user",user_template)
        ])

chain = chat_prompt	| bedrock_llm
chain.invoke({"text":"What's the capital of the USA?"})

AIMessage(content='The capital of the United States of America is Washington, D.C. It is a federal district located on the East Coast between the states of Maryland and Virginia. Washington, D.C. serves as the seat of the federal government and is home to many national landmarks and museums.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '898100b4-9303-4b72-bd18-034cc6e48e73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 22 Sep 2024 09:10:05 GMT', 'content-type': 'application/json', 'content-length': '455', 'connection': 'keep-alive', 'x-amzn-requestid': '898100b4-9303-4b72-bd18-034cc6e48e73'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 1137}}, id='run-ab6def6b-fdc0-441a-9a5d-d1d28690ba03-0', usage_metadata={'input_tokens': 37, 'output_tokens': 60, 'total_tokens': 97})

## Trulens

In [2]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

🦑 TruSession initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


* Bedrock llm - trulens lib

In [6]:
from trulens.providers.bedrock import Bedrock

bedrock = Bedrock(model_id=model_id, region_name=region_name)


* Initialize Feedback Function(s)

In [7]:
from trulens.core import Feedback
from trulens.core import Select
import numpy as np



In [8]:
# Question/answer relevance between overall question and answer.
f_anwer_relevance = Feedback(
    bedrock.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


* Instrument chain for logging with TruLens

In [9]:
from trulens.apps.langchain import TruChain

tru_recorder = TruChain(
    chain, app_name="Chain", feedbacks=[f_anwer_relevance]
)

* Run the chain with real examples

In [10]:
with tru_recorder as recording:
    llm_response = chain.invoke({"text":"What's the capital of the USA?"})

* Explore results in the notebook

In [13]:
session.get_records_and_feedback()[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Answer Relevance_calls,Answer Relevance feedback cost in USD,app_name,app_version,latency,total_tokens,total_cost,cost_currency
0,app_hash_0172c1f19152704135a2310314894050,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_1737a7d84371c81c27dbd36e95d705d3,"""What's the capital of the USA?""","""The capital of the United States of America i...",-,"{""record_id"": ""record_hash_1737a7d84371c81c27d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-09-22T11:10:11.231009"", ""...",2024-09-22T11:10:18.228145,1.0,[{'args': {'prompt': 'What's the capital of th...,0.0,Chain,base,6,0,0.0,USD


*  Explore in a Dashboard

In [14]:
session.get_leaderboard()

,,Answer Relevance,latency,total_cost
app_name,app_version,,,
Chain,base,1.0,6.0,0.0
